# Example of 

In [ ]:
from sklearn.datasets import load_iris

X = load_iris().data

In [1]:
from asid.automl_small.gm import GenerativeModel

In [ ]:
clf = GenerativeModel(gen_model_type="sklearn_kde", num_syn_samples=10, hyperopt_time=3)
clf.fit(X)